# Import librairies & data

In [129]:
import pandas as pd
import matplotlib.pyplot as plt
import plotly
import matplotlib.colors as mcolors
import plotly.express as px

In [2]:
df = pd.read_csv("bench.csv")
df['lookahead'] = df['lookahead'].fillna(0).astype(int)
df_group = df.groupby(['m', 'n', 'k', 'b', 'p', 'q', 'algo', 'lookahead']).agg({'gflops': 'mean'}).reset_index()

In [3]:
print("Shape:", df_group.shape)
df_group.head()

Shape: (810, 9)


,m,n,k,b,p,q,algo,lookahead,gflops
0,512,512,512,128,1,1,bcast,8,9.903908
1,512,512,512,128,1,1,p2p,8,11.825650
2,512,512,512,128,1,1,p2p-i-la,1,10.092977
3,512,512,512,128,1,1,p2p-i-la,2,9.677004
4,512,512,512,128,1,1,p2p-i-la,3,8.801812


# Exploration de bench.csv

In [4]:
print(set(df_group['m']))
print(set(df_group['n']))
print(set(df_group['k']))
print('--------------------')
print(set(df_group['b']))
print('--------------------')
print(set(df_group['p']))
print(set(df_group['q']))
print('--------------------')
print(set(df_group['lookahead']))
print(set(df_group['algo']))


{512, 1024, 3072, 2048, 1536, 4096, 6144}
{512, 1024, 3072, 2048, 1536, 4096, 6144}
{512, 1024, 3072, 2048, 1536, 4096, 6144}
--------------------
{128, 256, 512}
--------------------
{1, 2, 4}
{1, 2, 4}
--------------------
{1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12}
{'bcast', 'p2p', 'p2p-i-la'}


# Analyse lookahead

In [5]:
df_p2p_i_la = df_group[df_group['algo'].str.contains('la')]
df_p2p_i_la_filtered = df_p2p_i_la.query('p == 2').query('q == 2')

In [34]:
cmap = plt.cm.get_cmap('plasma', 12)
colors = [mcolors.to_hex(cmap(i)) for i in range(cmap.N)]

/tmp/ipykernel_5556/1879595275.py:1: MatplotlibDeprecationWarning:

The get_cmap function was deprecated in Matplotlib 3.7 and will be removed in 3.11. Use ``matplotlib.colormaps[name]`` or ``matplotlib.colormaps.get_cmap()`` or ``pyplot.get_cmap()`` instead.



In [36]:
plotly.plot(df_p2p_i_la_filtered.query('b == 256').sort_values(['n', 'p', 'q', 'lookahead']), kind = 'line', x =  'n', y = 'gflops', color = 'lookahead', markers = True, color_discrete_sequence=colors, title = 'Evolution des Gflops en fonction de n pour différentes valeurs de lookahead (b=256)') #, facet_col="p", facet_row="q")

In [9]:
plotly.plot(df_p2p_i_la_filtered, kind = 'line', x =  'lookahead', y = 'gflops', height = 800, color = 'n', facet_row = 'b', markers = True, title = 'Evolution des Gflops en fonction du paramètre lookahead pour différentes valeurs de b et avec q = p = 2')

## Trouver le meilleur lookahead pour chaque combinaison

In [10]:
idx_max = df_p2p_i_la.groupby(['m', 'n', 'k', 'b', 'p', 'q'])['gflops'].idxmax()
df_p2p_i_la_max = df_group.loc[idx_max]
df_others = df_group[df_group['algo'] != 'p2p-i-la']
df_filtered_best_lookahead = pd.concat([df_others, df_p2p_i_la_max], ignore_index=True)

In [ ]:
fig = px.density_heatmap(
    df_p2p_i_la_max.query('b == 512 and n == 6144'),
    x='p',
    y='q',
    z='lookahead',
    text_auto=True,
    color_continuous_scale='Viridis', 
    title='Heatmap du meilleur lookahead pour b=512, n=6144 selon différentes valeurs de p et de q'
)

fig.update_layout(
    xaxis_title="p",
    yaxis_title="q"
)

fig.show()

# Analyse de b

## p = q = 1

In [12]:
df_p_1 = df_filtered_best_lookahead.sort_values(["m", "b", "p", "q"]).query('p == 1').query('q == 1')

In [13]:
plotly.plot(df_p_1, kind = 'line', markers = True, x = 'n', y = 'gflops', color ='algo', facet_col = 'b', width = 1400, title = 'Evolution des Gflops en fonction de la taille du problème pour différentes valeurs de b et pour différents algorithmes, p = q = 1').update_xaxes(matches=None)

## p = q = 2

In [14]:
df_p_2 = df_filtered_best_lookahead.sort_values(["m", "b", "p", "q"]).query('p == 2').query('q == 2')

In [15]:
plotly.plot(df_p_2, kind = 'line', markers = True, x = 'n', y = 'gflops', color ='algo', facet_col = 'b', width = 1400, title = 'Evolution des Gflops en fonction de la taille du problème pour différentes valeurs de b et pour différents algorithmes, p = q = 2').update_xaxes(matches=None)

## p = q = 4

In [16]:
df_p_4 = df_filtered_best_lookahead.sort_values(["m", "b", "p", "q"]).query('p == 4').query('q == 4')

In [17]:
plotly.plot(df_p_4, kind = 'line', markers = True, x = 'n', y = 'gflops', color ='algo', facet_col = 'b', width = 1400, title = 'Evolution des Gflops en fonction de la taille du problème pour différentes valeurs de b et pour différents algorithmes, p = q = 4').update_xaxes(matches=None)

## p = 1 et q = 2

In [18]:
df_p1_q2 = df_filtered_best_lookahead.sort_values(["m", "b", "p", "q"]).query('p == 1').query('q == 2')

In [19]:
plotly.plot(df_p1_q2, kind = 'line', markers = True, x = 'n', y = 'gflops', color ='algo', facet_col = 'b', width = 1400, title = 'Evolution des Gflops en fonction de la taille du problème pour différentes valeurs de b et pour différents algorithmes, p = 1 et q = 2').update_xaxes(matches=None)

# Analyse de p et q avec variation de m, n et k

## b = 128

In [20]:
df_b128 = df_filtered_best_lookahead.sort_values(["m", "b", "p", "q"]).query('b == 128')

In [21]:
plotly.plot(df_b128, kind = 'line', markers = True, x = 'n', y = 'gflops', color ='algo', facet_col = 'p', facet_row = 'q', height = 700, width = 1400, title = 'Evolution des Gflops pour différentes valeurs de p et q avec b = 128')

## b = 256

In [22]:
df_b256 = df_filtered_best_lookahead.sort_values(["m", "b", "p", "q"]).query('b == 256')

In [24]:
plotly.plot(df_b256, kind = 'line', markers = True, x = 'n', y = 'gflops', color ='algo', facet_col = 'p', facet_row = 'q', height = 700, width = 1400, title = 'Evolution des Gflops pour différentes valeurs de p et q avec b = 256')

## b = 512

In [25]:
df_b512 = df_filtered_best_lookahead.sort_values(["m", "b", "p", "q"]).query('b == 512')

In [26]:
plotly.plot(df_b512, kind = 'line', markers = True, x = 'n', y = 'gflops', color ='algo', facet_col = 'p', facet_row = 'q', height = 700, width = 1400, title = 'Evolution des Gflops pour différentes valeurs de p et q avec b = 512')

# Analyse de p et q sans variation de m, n et k

In [27]:
df_filtered_best_lookahead.query('b == 256').query('n == 1024')

,m,n,k,b,p,q,algo,lookahead,gflops
36,1024,1024,1024,256,1,1,bcast,12,17.109553
37,1024,1024,1024,256,1,1,p2p,12,21.619357
38,1024,1024,1024,256,1,2,bcast,12,19.260844
39,1024,1024,1024,256,1,2,p2p,12,22.068472
40,1024,1024,1024,256,1,4,bcast,12,21.426918
41,1024,1024,1024,256,1,4,p2p,12,24.636784
42,1024,1024,1024,256,2,1,bcast,12,22.764214
43,1024,1024,1024,256,2,1,p2p,12,19.535253
44,1024,1024,1024,256,2,2,bcast,12,23.405053
45,1024,1024,1024,256,2,2,p2p,12,30.139974


In [28]:
df_filtered_best_lookahead.query('b == 256').query('n == 1024')

,m,n,k,b,p,q,algo,lookahead,gflops
36,1024,1024,1024,256,1,1,bcast,12,17.109553
37,1024,1024,1024,256,1,1,p2p,12,21.619357
38,1024,1024,1024,256,1,2,bcast,12,19.260844
39,1024,1024,1024,256,1,2,p2p,12,22.068472
40,1024,1024,1024,256,1,4,bcast,12,21.426918
41,1024,1024,1024,256,1,4,p2p,12,24.636784
42,1024,1024,1024,256,2,1,bcast,12,22.764214
43,1024,1024,1024,256,2,1,p2p,12,19.535253
44,1024,1024,1024,256,2,2,bcast,12,23.405053
45,1024,1024,1024,256,2,2,p2p,12,30.139974


In [128]:
# fig = px.box(
#     df_filtered_best_lookahead.query('b == 256').query('n == 1024'),
#     x='p',
#     y='gflops',
#     color='algo',
#     title="Evolution des Gflops pour différents algorithmes avec b=256 et n=1024",
#     points="all" 
# )


# fig.show()

# Influence de b et de l'algo 

In [38]:
fig = px.box(
    df_filtered_best_lookahead,
    x='b',
    y='gflops',
    color='algo',
    title="Influence de la taille de la matrice b et de l'algorithme utilisé sur les gflops",
    points="outliers" 
)

fig.update_layout(
    boxmode='group', 
    width=800,
    height=500
)

fig.show()


# Matrice de corrélation

In [31]:
params = ['m', 'n', 'k', 'b', 'p', 'q', 'lookahead']
corr = df_filtered_best_lookahead[params + ['gflops']].corr().round(2)

corr_long = corr.reset_index().melt(id_vars='index')
corr_long.columns = ['param_x', 'param_y', 'correlation']

fig = px.imshow(
    corr.values,
    x=corr.columns,
    y=corr.columns,
    text_auto=True,
    color_continuous_scale='RdBu',
    zmin=-1,
    zmax=1,
    title="Matrice de corrélation entre les paramètres et GFLOPS"
)

fig.update_layout(
    width=600,
    height=500
)

fig.show()


# Affichage pour différentes platformes

In [132]:
df_crossbar = pd.read_csv("bench_crossbar.csv")
df_crossbar['lookahead'] = df_crossbar['lookahead'].fillna(0).astype(int)
df_crossbar_group = df_crossbar.groupby(['m', 'n', 'k', 'b', 'p', 'q', 'algo', 'lookahead']).agg({'gflops': 'mean'}).reset_index().assign(platform = 'cluster_barcross')
df_fat_tree = pd.read_csv("bench_fat_tree.csv")
df_fat_tree['lookahead'] = df_fat_tree['lookahead'].fillna(0).astype(int)
df_fat_tree_group = df_fat_tree.groupby(['m', 'n', 'k', 'b', 'p', 'q', 'algo', 'lookahead']).agg({'gflops': 'mean'}).reset_index().assign(platform = 'cluster_fat_tree')

In [140]:

df_all = pd.concat([df_crossbar_group, df_fat_tree_group])

fig = px.box(
    df_all,
    x='n',            
    y='gflops',     
    color='platform',  
    points='all',     
    title='Performance en GFLOPS par platform selon différentes tailles avec b = 256 et p = q = 2',
    labels={'n': 'Taille (n)', 'gflops': 'GFLOPS'}
)

fig.show()
